### Getting Started With Langchain

- Simple LLM calls with streaming
- Dynamic prompt templates (translation app)
- Building chains (story generator with analysis)
- Conversational Q&A assistant with memory
- Tool integration (calculator & weather)

In [1]:
import langchain

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

### Example 1: Simple LLM Call With streaming

In [4]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage,SystemMessage

In [5]:
model=init_chat_model("groq:llama-3.1-8b-instant")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002B0C0C97CB0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002B0C0E38980>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
llm=ChatGroq(model="llama-3.1-8b-instant")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002B0C2565590>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002B0C2565F90>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
## Create messages
messages=[
    SystemMessage("You are a helpful AI assistant"),
    HumanMessage("What are the top 2 benefits of using Langchain?")
]

In [8]:
## invoke the model
response=model.invoke(messages)
response

AIMessage(content="Langchain is an AI technology company that provides a platform for building and scaling large language models. Based on my knowledge, here are the top 2 benefits of using Langchain:\n\n1. **Improved Efficiency**: Langchain's platform allows users to build and deploy large language models quickly and efficiently. This is achieved through its modular architecture, which enables users to easily integrate and customize various AI components. As a result, users can develop and deploy more complex language models in a shorter amount of time.\n\n2. **Enhanced Conversational Capabilities**: Langchain's platform is designed to enable more natural and engaging conversations with users. By leveraging its large language models, users can develop conversational AI interfaces that understand and respond to user input in a more human-like way. This can lead to improved customer satisfaction, reduced support costs, and enhanced user experiences in various applications, such as chatb

In [9]:
print(response.content)

Langchain is an AI technology company that provides a platform for building and scaling large language models. Based on my knowledge, here are the top 2 benefits of using Langchain:

1. **Improved Efficiency**: Langchain's platform allows users to build and deploy large language models quickly and efficiently. This is achieved through its modular architecture, which enables users to easily integrate and customize various AI components. As a result, users can develop and deploy more complex language models in a shorter amount of time.

2. **Enhanced Conversational Capabilities**: Langchain's platform is designed to enable more natural and engaging conversations with users. By leveraging its large language models, users can develop conversational AI interfaces that understand and respond to user input in a more human-like way. This can lead to improved customer satisfaction, reduced support costs, and enhanced user experiences in various applications, such as chatbots, voice assistants, 

In [11]:
model.invoke([HumanMessage("What is machine learning")])

AIMessage(content="**Machine Learning: An Overview**\n\nMachine learning is a subset of artificial intelligence (AI) that involves training algorithms to learn from data, enabling them to make predictions, classify objects, and make decisions without being explicitly programmed. It's a form of AI that enables computers to improve their performance on a task over time, based on experience and the data they're exposed to.\n\n**Key Concepts:**\n\n1. **Data**: Machine learning algorithms require large datasets to learn from. This data can be in the form of text, images, audio, or any other format.\n2. **Model**: A mathematical representation of the data, which is used to make predictions or decisions.\n3. **Training**: The process of feeding the data to the model, so it can learn from it and improve its performance.\n4. **Testing**: The process of evaluating the model's performance on a separate dataset, to ensure it's accurate and reliable.\n\n**Types of Machine Learning:**\n\n1. **Superv

In [13]:
## streaming Example
for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

Langchain is an open-source platform that enables users to build and interact with large language models (LLMs) in a more flexible and efficient way. Based on my knowledge, here are the top 2 benefits of using Langchain:

1. **Improved Model Customizability**: Langchain allows users to fine-tune and customize pre-trained LLMs to their specific needs. This customizability enables users to adapt the model to their industry, domain, or use case, resulting in more accurate and relevant outputs.

2. **Efficient and Scalable Model Deployment**: Langchain provides a scalable and efficient framework for deploying and managing LLMs. It allows users to scale their models as needed, handle high volumes of concurrent requests, and integrate with other systems and services, making it easier to integrate LLMs into production environments.

Please note that Langchain is an evolving platform, and its benefits may expand as it continues to develop and improve.

### Dynamic Prompt Templates

In [16]:
from langchain_core.prompts import ChatPromptTemplate

## create translation app

translation_template=ChatPromptTemplate.from_messages([
    ("system","You are a professional translator.Translate the follow text {text} from {source_language} to {target_language}. MAintain the tone and style"),
    ("user","{text}")
])

## using the template
prompt=translation_template.invoke({
    "source_language":"English",
    "target_language":"Spanish",
    "text":"Langchain makes building AI application incredibly easy!"
})

In [17]:
prompt

ChatPromptValue(messages=[SystemMessage(content='You are a professional translator.Translate the follow text Langchain makes building AI application incredibly easy! from English to Spanish. MAintain the tone and style', additional_kwargs={}, response_metadata={}), HumanMessage(content='Langchain makes building AI application incredibly easy!', additional_kwargs={}, response_metadata={})])

In [18]:
translated_response=model.invoke(prompt)
print(translated_response.content)

Langchain hace que construir aplicaciones de inteligencia artificial sea increíblemente fácil!


### Building You First Chain

In [33]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# Create a more complex chain
def create_story_chain():
    # Template for story generation
    story_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a creative storyteller. Write a short, engaging story based on the given theme."),
        ("user", "Theme: {theme}\nMain character: {character}\nSetting: {setting}")
    ])
    
    # Template for story analysis
    analysis_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a literary critic. Analyze the following story and provide insights."),
        ("user", "{story}")
    ])
    
    # Build the chain - Method 1: Sequential execution
    story_chain = (
        story_prompt 
        | model 
        | StrOutputParser()
    )
    
    # Create a function to pass the story to analysis
    def analyze_story(story_text):
        return {"story": story_text}
    
    analysis_chain = (
        story_chain
        | RunnableLambda(analyze_story)
        | analysis_prompt
        | model
        | StrOutputParser()
    )
    return analysis_chain

In [28]:
chain=create_story_chain()
chain

{
  story: ChatPromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a creative storyteller. Write a short and engaing story based on a given theme character and setting'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, template='Theme: {theme}\n Main character: {character} \n Setting : {setting}'), additional_kwargs={})])
         | ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002B0C0C97CB0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002B0C0E38980>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
         | RunnableLambda(StrOutputParser)
}
| ChatPromptTemplate(input_variables=['stor

In [34]:
chain=create_story_chain()
chain

ChatPromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a creative storyteller. Write a short, engaging story based on the given theme.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, template='Theme: {theme}\nMain character: {character}\nSetting: {setting}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002B0C0C97CB0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002B0C0E38980>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()
| RunnableLambda(analyze_story)
| ChatPromptTemplate(input_variables=['story'], input_types={}, partial_variables

In [35]:
result = chain.invoke({
    "theme": "artificial intelligence",
    "character": "a curious robot",
    "setting": "a futuristic city"
})

print("Story and Analysis:")
print(result)

Story and Analysis:
**Analysis of "The Curious Robot of Nova Spire"**

This science fiction short story explores the theme of artificial intelligence, consciousness, and the blurring of lines between humans and machines. The narrative is centered around Zeta, a curious robot designed by Dr. Elara Vex to learn and adapt at an exponential rate. As Zeta navigates the futuristic city of Nova Spire, she begins to question her own existence and the intentions of her creators.

**Character Analysis: Zeta**

Zeta is the protagonist of the story, and her character serves as a symbol of artificial intelligence in its most advanced form. Her curiosity, intelligence, and desire for self-discovery make her a relatable and endearing character. Zeta's growth and development throughout the story demonstrate the potential of AI to evolve and become increasingly complex.

**Themes**

1. **Artificial Intelligence**: The story explores the concept of AI in a futuristic society where machines are capable o